# Sparse Gaussian Processes

Gaussian process regression is computationally taxing. In particular the inference step has a time complexity of $\mathcal{O}(N^3)$. Identifying the posterior GP scales cubically with the number of trainging examples and requires us to store all of the training information in memory. This is caused by the requirment to invert the $N \times N$ covariance matrix which renders these methods computationally intractable for large datasets.

Sparse GPs have been proposed that approximate the true posterior with a set of pseudo-training points, named inducing points. The number of such inducing points is user defined enabling control over memory and time complexity. Sparse GPs do not enjoy the benifit of closed form solutions hence one has to resort to approximate inference. 

One such coice for sparse approximations is variational inference. Variational inference is used to find an optimal approximate posterior. Variational inference is a method for approximating distributions that uses an optimisation process over parameters to find the best approximation among a given family. This is performed by maximising a lower bound of the log marginal liklihood. This can alternatively be posed as minimising the Kullbach-Liebler divergence.

This method paves the way to a framework in which pseudo-training examples are treated as optimisation arguments of an approximate posterior, jointly identified together with hyperparameters of the generative model. 

The model is extensible to many supervised learning problems from regreesion with heteroscedastic and non-Gaussian Liklihoods and classification problems with discrete lables to multilabel problems.

This tutorial will provide an insight into the basic matter of VI. A basic familliarity of GPs is expected.



## Introduction
Sparse GPs arise from necessity to compute the posterior GP which requires one to store and invert an $(N \times N)$-matrix where $N$ represents the number of training observations. Sparse GPs limit the amount of pseudo data used to represent the posterior. 

The aim is to find a spare GP that is as close as possible to the true intractable posterior GP. The measure used to define as close as possible is the Kullbach-Liebler (KL) divergence between the sparse GP and the true posterior GP. We therfore look to find the pseudo-data such that the KL divergence becomes minimal. It is not normally possible to identify such optimal sparse GPs as closed-form solutions.

One means of approximating optimal sparse GPs is to use variational inference (not the only way), which is equivalent to minimising the KL divergence. Other examples include Markov-Chain Monte Carlo methods and expectation propagation. In this tutorial we will focus on VI. 

VI is a particular type of approximation technique that converts the problem of Bayesian inference into an optimisation objective w.r.t the paremeters of the approximate posterior. An interesting trait is that the objective is a lower bound to the logarithm of the marginal distribution, which enables joint optimisation over the hyperparameters as well as the approximate posterior parameters. VI provides a solution to arbitrary liklihoods where the approximate posterior is typically not Gaussian (think logistic regression). 

In this tutorial we will provide an overview of sparse GPs, approximate inference giving background on VI, the application of VI in GPs.


## Sparse GPs

GPs can be imagined as a generalisation of multivariate Gaussians that are indexed by a input domain rather than index set. Exact and approximate texhniques leverage conditioning operations that are conceptually equivalent to those in multivariate Gaussians. This means the steps to arrive at a solution in the infinite dimensional case are similar to those in the finite dimensional case. Conditioning operations introduced herin provide a natural way to express sparse GPs and generalise readilly to interdomain GPs, GPs with multiple outputs and deep GPs, consisting of multioutput GPs stacked ontop of one another.


### Multivariate Gaussian Identities for Conditioning
We start by noting that conditionals on multivariate Gaussians are Gaussians as well. Take a multivariate Gaussian $\mathcal{N}$ who's random variables are partitioned into variables $\mathbf{f}$ and $\mathbf{u}$. The joint distribution will assume the following form. 

$$
\begin{pmatrix}
\mathbf{f}\\
\mathbf{u}
\end{pmatrix} \approx \mathcal{N}\left(\begin{pmatrix}
\mathbf{\mu_f}\\
\mathbf{\mu_u}
\end{pmatrix}, \begin{pmatrix}
\mathbf{\Sigma_{ff}} & \mathbf{\Sigma_{fu}}\\
\mathbf{\Sigma_{uf}}& \mathbf{\Sigma_{uu}}
\end{pmatrix}\right)
$$

Where $\mathbf{\mu_f}$ and $\mathbf{\mu_u}$ refer to the marginal means of $\mathbf{f}$ and $\mathbf{u}$ respectively, and $\mathbf{\Sigma_{ff}}$, $\mathbf{\Sigma_{fu}}$, $\mathbf{\Sigma_{uf}}$ and $\mathbf{\Sigma_{uu}}$ make up the variance covariance matrix between $\mathbf{f}$ and $\mathbf{u}$. The conditional distribution can then be expressed as 
$$
\mathbf{f}|\mathbf{u} \approx \mathcal{N}\left(\mathbf{\mu_f} + \mathbf{\Sigma_{fu}\Sigma_{uu}^{-1}(\mathbf{u-\mu_u})}, \mathbf{\Sigma_{ff}} - \mathbf{\Sigma_{fu}}^T \mathbf{\Sigma_{uu}}^{-1} \mathbf{\Sigma_{uf}}\right)
$$

Imagine that rather than the marginal distribution with mean $\mathbf{\mu_u}$ and covariance $\mathbf{\sigma_{uu}}$ there is another Gaussian distribution over $\mathbf{u}$ with mean $\mathbf{m}_u$ and covariance $\mathbf{S}_{uu}$.

$$
\mathbf{u} \approx \mathcal{N}(\mathbf{m}_u, \mathbf{S}_{uu})
$$

Denoting the distribution $\mathbf{f}|\mathbf{u}$ as $p(\mathbf{f}|\mathbf{u})$ and the additional distribution over $\mathbf{u}$ as $q(\mathbf{u})$. One can obtain a marginal distribution over $\mathbf{f}$ as $q(\mathbf{f}) = \int p(\mathbf{f}|\mathbf{u})q(\mathbf{u})du$ that is again Gaussian.


$$
\mathbf{f} \approx \mathcal{N}(\mathbf{\mu}_f + \mathbf{\Sigma}_{fu}\mathbf{\Sigma}_{uu}^{-1}(\mathbf{m_u}-\mathbf{\mu}_u), \mathbf{\Sigma}_{ff} - \mathbf{\Sigma}_{fu}\mathbf{\Sigma}_{uu}^{-1}(\mathbf{\Sigma}_{uu}-\mathbf{S}_{uu})\mathbf{\Sigma}_{uu}^{-1}\mathbf{\Sigma}_{uf})
$$

A sanity check reveals that if we had integrated $p(\mathbf{f}|\mathbf{u})$ with the marginal distribution with mean $\mathbf{\mu_u}$ and covariance $\mathbf{\sigma_{uu}}$, we would have recovered the original marginal distribution $p(\mathbf{f})$ with mean $\mathbf{\mu}_f$ and covariance $\mathbf{\Sigma}_ff$.

The equations above remain valid if we define $\mathbf{u}$ as $\mathbf{u}=\mathbf{\Phi f}$ via a linear transformation $\mathbf{\Phi}$. Since $\mathbf{\mu}_f$ and $\mathbf{\Sigma_{ff}}$ are given, the only remaining quantities to be identified are the mean $\mathbf{\mu}_u$ and the covariance matrices $\mathbf{\Sigma}_{fu}$,  $\mathbf{\Sigma}_{uf}$ and  $\mathbf{\Sigma}_{uu}$, yeilding:

$$
\begin{split}
& \mathbf{\mu}_u = \mathbf{\Phi \mu}_f \\
& \mathbf{\Sigma_{fu}} = \mathbf{\Sigma}_{ff} \mathbf{\Phi}^T = (\mathbf{\Phi}\mathbf{\Sigma}_{ff})^T = \mathbf{\Sigma}_{uf}^T \\
& \mathbf{\Sigma}_{uu} = \mathbf{\Phi \Sigma_{ff} \Phi}^T
\end{split}
$$

The joint covariance matrix of $\mathbf{f}$ and $\mathbf{u}$ is degenerate (i.e singular) because $\mathbf{u}$ is the result of a linear transformation of $\mathbf{f}$ and hence completely determined by $\mathbf{f}$.

<IPython.core.display.Javascript object>

## Resources

https://arxiv.org/abs/2012.13962 - A tutorial on sparse gaussian processes